In [17]:
import pandas as pd
import numpy as np
import os
from datasets import load_dataset, Audio
import shutil
import soundfile as sf
from tqdm import tqdm
from IPython.display import Audio, display

In [2]:
os.getcwd()

'/home/jovyan/work/Datasets'

In [4]:
from huggingface_hub import login

In [ ]:
login(token="HF-TOKEN")  # Replace with your token

In [6]:
from datasets import load_dataset
bhasaanuvaad = load_dataset("ai4bharat/IndicVoices-ST", "indic2en", split="telugu")

train-00000-of-00002.parquet:   0%|          | 0.00/1.58G [00:00<?, ?B/s]

train-00001-of-00002.parquet:   0%|          | 0.00/1.59G [00:00<?, ?B/s]

train-00000-of-00002.parquet:   0%|          | 0.00/1.38G [00:00<?, ?B/s]

train-00001-of-00002.parquet:   0%|          | 0.00/1.40G [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/164M [00:00<?, ?B/s]

...

Generating punjabi split:   0%|          | 0/71933 [00:00<?, ? examples/s]

Generating nepali split:   0%|          | 0/121852 [00:00<?, ? examples/s]

Generating tamil split:   0%|          | 0/235647 [00:00<?, ? examples/s]

Generating telugu split:   0%|          | 0/160355 [00:00<?, ? examples/s]

Generating urdu split:   0%|          | 0/115034 [00:00<?, ? examples/s]

In [7]:
print(bhasaanuvaad[0].keys())

dict_keys(['chunked_audio_filepath', 'text', 'pred_text', 'audio_filepath', 'start_time', 'duration', 'alignment_score', 'en_text', 'en_mining_score'])


In [8]:
print(bhasaanuvaad)

Dataset({
    features: ['chunked_audio_filepath', 'text', 'pred_text', 'audio_filepath', 'start_time', 'duration', 'alignment_score', 'en_text', 'en_mining_score'],
    num_rows: 160355
})


In [9]:
print(bhasaanuvaad.column_names)

['chunked_audio_filepath', 'text', 'pred_text', 'audio_filepath', 'start_time', 'duration', 'alignment_score', 'en_text', 'en_mining_score']


In [10]:
print(bhasaanuvaad[0])

{'chunked_audio_filepath': {'path': '0.wav', 'array': array([-0.00122881, -0.00187302, -0.00195422, ..., -0.00363457,
       -0.00297233, -0.00254051], shape=(56320,)), 'sampling_rate': 16000}, 'text': 'కొన్ని పండుగ వస్తువులు ప్రాడక్టులు సూచించు', 'pred_text': 'కొన్ని పండుగ వస్తువులు ప్రాడక్టులు సూచించు', 'audio_filepath': '/data-4/sparsh/shrutilipi-pipeline/NPTEL/spoken_output/indicvoices_output/final_audios/telugu/4503599627491103.wav', 'start_time': 1.04, 'duration': 3.52, 'alignment_score': 1.0, 'en_text': 'Some festival items and products are suggested', 'en_mining_score': 0.6999157071113586}


**From extraction to saving final csv pipeline**

In [16]:
class IndicVoiceProcessor:
    def __init__(self, dataset_name, language, audio_column, text_column, output_audio_dir, output_csv_path, final_audio_folder):
        self.dataset = load_dataset(dataset_name, "indic2en", split=language)
        self.dataset = self.dataset.cast_column(audio_column, Audio())
        self.audio_column = audio_column
        self.text_column = text_column
        self.output_audio_dir = output_audio_dir
        self.output_csv_path = output_csv_path
        self.final_audio_folder = final_audio_folder
        os.makedirs(self.output_audio_dir, exist_ok=True)
        os.makedirs(self.final_audio_folder, exist_ok=True)

    def extract_and_save_audio(self):
        records = []

        if os.listdir(self.output_audio_dir): # Check and empty the folder if not empty
            print(f" Folder '{self.output_audio_dir}' is not empty. Clearing contents...")
            for file in os.listdir(self.output_audio_dir):
                file_path = os.path.join(self.output_audio_dir, file)
                if os.path.isfile(file_path):
                    os.remove(file_path)
            print(" Folder cleared.")

        print("Extracting and saving audio files...")
        for idx, example in tqdm(enumerate(self.dataset), total=len(self.dataset)):
            audio_data = example[self.audio_column]
            audio_array = audio_data["array"]
            sample_rate = audio_data["sampling_rate"]
            duration_sec = len(audio_array) / sample_rate
            duration_ms = duration_sec * 1000
            audio_filename = f"{idx}.wav"
            audio_path = os.path.join(self.output_audio_dir, audio_filename)
            sf.write(audio_path, audio_array, sample_rate)

            records.append({
                "audio_filename": audio_filename,
                "text": example[self.text_column],
                "duration": example["duration"],
                "duration_sec": duration_sec,  # No rounding
                "duration_ms": duration_ms    # No rounding
            })

        df = pd.DataFrame(records)
        return df

    def validate_audio_files(self, df):
        print("Validating existence of audio files...")
        available_files = set(os.listdir(self.output_audio_dir))
        df['found'] = df['audio_filename'].apply(lambda x: 'yes' if x in available_files else 'no')
        return df

    def remove_duplicates(self, df):
        print("Removing duplicates...")
        df_unique = df.drop_duplicates(subset=["audio_filename"])
        return df_unique

    def rename_and_reorder_columns(self, df):
        print("Renaming and reordering columns...")
        df.rename(columns={
            "audio_filename": "path",
            "text": "sentence"
        }, inplace=True)

        # Move 'found' column after 'sentence'
        cols = list(df.columns)
        found_idx = cols.index("found")
        cols.insert(cols.index("sentence") + 1, cols.pop(found_idx))  # move 'found' after 'sentence'
        df = df[cols]
        return df

    def remove_large_duration_rows(self, df):
        print("Removing rows with duration_sec > 15...")
        df_filtered = df[df["duration_sec"] <= 15]
        return df_filtered

    def check_and_copy_audio_files(self, df):
        print("Checking and copying audio files to final folder...")
        for idx, row in tqdm(df.iterrows(), total=df.shape[0]):
            audio_filename = row["path"]
            audio_path = os.path.join(self.output_audio_dir, audio_filename)
            if os.path.exists(audio_path):
                shutil.copy(audio_path, self.final_audio_folder)

    def drop_duration_column_if_same(self, df):
        if df["duration"].equals(df["duration_sec"]):
            print("Dropping 'duration' column as it's the same as 'duration_sec'...")
            df.drop(columns=["duration"], inplace=True)
        return df

    def generate_summary(self, df):
        print("\n DataFrame Summary:")
        print(df.info())
        print("\n Describe:")
        print(df.describe())
        print("\n Head:")
        print(df.head())
        print("\n Tail:")
        print(df.tail())
        print("\n Unique Audio Files:", df['path'].nunique())

        # Calculate the total durations
        total_duration = df['duration'].sum()
        total_duration_sec = df['duration_sec'].sum()
        total_duration_ms = df['duration_ms'].sum()
        print(f"\nTotal Duration (sec): {total_duration}")
        print(f"Total Duration (ms): {total_duration_ms}")
        print(f"Total Duration (seconds): {total_duration_sec}")

    def run_pipeline(self):
        df = self.extract_and_save_audio()
        df = self.validate_audio_files(df)
        df = self.remove_duplicates(df)
        df = self.rename_and_reorder_columns(df)
        df = self.remove_large_duration_rows(df)

        # Save filtered CSV
        df.to_csv(self.output_csv_path, index=False)

        # Create final folder and copy audio files
        self.check_and_copy_audio_files(df)

        # Drop duration column if 'duration' and 'duration_sec' are the same
        df = self.drop_duration_column_if_same(df)

        # Save the filtered file
        df.to_csv("/home/jovyan/work/Datasets/filtered_telugu_BA.csv", index=False)

        # Generate summary and statistics
        self.generate_summary(df)

        print(f"\n Final CSV saved at: {self.output_csv_path}")


# === Run the processor in Jupyter ===
processor = IndicVoiceProcessor(
    dataset_name="ai4bharat/IndicVoices-ST",
    language="telugu",
    audio_column="chunked_audio_filepath",
    text_column="text",
    output_audio_dir="/home/jovyan/work/Datasets/telugu_audio_chunks",
    output_csv_path="/home/jovyan/work/Datasets/filtered_telugu_dataset.csv",
    final_audio_folder="/home/jovyan/work/Datasets/telugu_audios_BA_final"
)

processor.run_pipeline()


 Folder '/home/jovyan/work/Datasets/telugu_audio_chunks' is not empty. Clearing contents...
 Folder cleared.
Extracting and saving audio files...


 66%|██████▋   | 106359/160355 [15:11<11:01, 81.59it/s]IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

100%|██████████| 160355/160355 [22:57<00:00, 116.38it/s]


Validating existence of audio files...
Removing duplicates...
Renaming and reordering columns...
Removing rows with duration_sec > 15...
Checking and copying audio files to final folder...


100%|██████████| 152656/152656 [01:35<00:00, 1606.17it/s]



 DataFrame Summary:
<class 'pandas.core.frame.DataFrame'>
Index: 152656 entries, 0 to 160354
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   path          152656 non-null  object 
 1   sentence      152656 non-null  object 
 2   found         152656 non-null  object 
 3   duration      152656 non-null  float64
 4   duration_sec  152656 non-null  float64
 5   duration_ms   152656 non-null  float64
dtypes: float64(3), object(3)
memory usage: 8.2+ MB
None

 Describe:
            duration   duration_sec    duration_ms
count  152656.000000  152656.000000  152656.000000
mean        5.259026       5.258832    5258.832218
std         3.167442       3.167436    3167.436330
min         0.080000       0.080000      80.000000
25%         2.800000       2.800000    2800.000000
50%         4.560000       4.560000    4560.000000
75%         7.040000       7.040000    7040.000000
max        15.040000      14.989000   14989.0

**For updated audio files folder**

In [22]:
audio_folder = "/home/jovyan/work/Datasets/telugu_audios_BA_final"
# List all .wav files
audio_files = [f for f in os.listdir(audio_folder) if f.endswith(".wav")]
# Count them
print(f" Total number of audio files: {len(audio_files)}")

 Total number of audio files: 152656


In [23]:
audio_files = sorted(os.listdir(audio_folder))[:5]  # first 5 files
for file in audio_files:
    print(f" Playing: {file}")
    display(Audio(filename=os.path.join(audio_folder, file)))

 Playing: 0.wav


 Playing: 1.wav


 Playing: 10.wav


 Playing: 100.wav


 Playing: 1000.wav


**For Original audio files folder**

In [21]:
audio_folder = "/home/jovyan/work/Datasets/telugu_audio_chunks"
# List all .wav files
audio_files = [f for f in os.listdir(audio_folder) if f.endswith(".wav")]
# Count them
print(f" Total number of audio files: {len(audio_files)}")

 Total number of audio files: 160355


In [25]:
audio_files = sorted(os.listdir(audio_folder))[:5]  # first 5 files

for file in audio_files:
    print(f" Playing: {file}")
    display(Audio(filename=os.path.join(audio_folder, file)))

 Playing: 0.wav


 Playing: 1.wav


 Playing: 10.wav


 Playing: 100.wav


 Playing: 1000.wav


**Dropping duration column**

In [26]:
import pandas as pd
csv_path = "/home/jovyan/work/Datasets/filtered_telugu_BA.csv" # Load the CSV file
df = pd.read_csv(csv_path)


print(f" Total columns before dropping: {df.shape[1]}") # Show total number of columns before dropping
print(" Column names:", df.columns.tolist())


if 'duration' in df.columns: # Drop the 'duration' column if it exists
    df.drop(columns=['duration'], inplace=True)
    print(" 'duration' column dropped.")
else:
    print(" 'duration' column not found.")


print(f" Total columns after dropping: {df.shape[1]}") # Show total columns after dropping
print(" Updated column names:", df.columns.tolist())


 Total columns before dropping: 6
 Column names: ['path', 'sentence', 'found', 'duration', 'duration_sec', 'duration_ms']
 'duration' column dropped.
 Total columns after dropping: 5
 Updated column names: ['path', 'sentence', 'found', 'duration_sec', 'duration_ms']


**Merging audio folders and csv files (telugu)**

In [34]:
# Audio file manager
class AudioDatasetManager:
    def __init__(self, folder_names, output_folder):
        self.folder_names = folder_names
        self.output_folder = output_folder

    def count_audio_files(self, folder):
        return len([file for file in os.listdir(folder) if file.endswith('.wav')])

    def process_audio_folders(self):
        print(" Counting and merging audio files...")
        total_files = {}
        os.makedirs(self.output_folder, exist_ok=True)

        for folder in self.folder_names:
            count = self.count_audio_files(folder)
            print(f"🎧 Total .wav files in '{folder}': {count}")
            total_files[folder] = count

            for file in os.listdir(folder):
                if file.endswith('.wav'):
                    src = os.path.join(folder, file)
                    dst = os.path.join(self.output_folder, file)
                    if not os.path.exists(dst):  # Avoid overwriting
                        shutil.copy2(src, dst)

        final_count = self.count_audio_files(self.output_folder)
        print(f"\n Total .wav files in merged folder '{self.output_folder}': {final_count}")
        return total_files, final_count


# CSV manager
class CSVDatasetManager:
    def __init__(self, file_names, output_file):
        self.file_names = file_names
        self.output_file = output_file

    def inspect_csv(self, file_path):
        df = pd.read_csv(file_path)
        print(f"\n File: {file_path}")
        print(f" Columns: {df.columns.tolist()}")
        print(f" Rows: {len(df)}")
        return df

    def merge_csvs(self):
        dataframes = []
        for path in self.file_names:
            df = self.inspect_csv(path)
            dataframes.append(df)

        final_df = pd.concat(dataframes, axis=0, ignore_index=True)
        final_df.to_csv(self.output_file, index=False)
        print(f"\n Merged CSV saved as '{self.output_file}'")
        return final_df

    def analyze_merged_csv(self, df):
        print("\n --- Merged CSV Analysis ---")
        print(f" Columns: {df.columns.tolist()}")
        print(f" Total Rows: {len(df)}")

        print("\n Info:")
        print(df.info())

        print("\n Describe:")
        print(df.describe())

        print("\n Head:")
        print(df.head())

        print("\n Tail:")
        print(df.tail())

        print("\n Unique values per column:")
        print(df.nunique())

        print("\n Total duration (seconds):", df['duration_sec'].sum())
        print(" Total duration (milliseconds):", df['duration_ms'].sum())

# AUDIO FILES
audio_folders = [
    "IndicTTS_Telugu/audio_final",
    "SLR66_Telugu/te_Telugu_merged",
    "telugu_audios_BA_final"
]
final_audio_output = "final_merged_telugu_audios"

audio_manager = AudioDatasetManager(audio_folders, final_audio_output)
audio_manager.process_audio_folders()

# CSV FILES
csv_files = [
    "filtered_with_durations_Indic_Telugu_cleaned.csv",
    "filtered_with_durations_openslr_telugu_cleaned.csv",
    "filtered_telugu_BA.csv"
]
final_csv_output = "final_merged_telugu_dataset.csv"

csv_manager = CSVDatasetManager(csv_files, final_csv_output)
merged_df = csv_manager.merge_csvs()
csv_manager.analyze_merged_csv(merged_df)


 Counting and merging audio files...
🎧 Total .wav files in 'IndicTTS_Telugu/audio_final': 5355
🎧 Total .wav files in 'SLR66_Telugu/te_Telugu_merged': 4448
🎧 Total .wav files in 'telugu_audios_BA_final': 152656

 Total .wav files in merged folder 'final_merged_telugu_audios': 162459

 File: filtered_with_durations_Indic_Telugu_cleaned.csv
 Columns: ['path', 'sentence', 'found', 'duration_sec', 'duration_ms']
 Rows: 5355

 File: filtered_with_durations_openslr_telugu_cleaned.csv
 Columns: ['path', 'sentence', 'found', 'duration_sec', 'duration_ms']
 Rows: 4448

 File: filtered_telugu_BA.csv
 Columns: ['path', 'sentence', 'found', 'duration', 'duration_sec', 'duration_ms']
 Rows: 152656

 Merged CSV saved as 'final_merged_telugu_dataset.csv'

 --- Merged CSV Analysis ---
 Columns: ['path', 'sentence', 'found', 'duration_sec', 'duration_ms', 'duration']
 Total Rows: 162459

 Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162459 entries, 0 to 162458
Data columns (total 6 columns):


**Dropping duration column from the final merged telugu csv**

In [35]:
# Load the CSV file
csv_path = "/home/jovyan/work/Datasets/final_merged_telugu_dataset.csv"
df = pd.read_csv(csv_path)

# Show total number of columns before dropping
print(f" Total columns before dropping: {df.shape[1]}")
print(" Column names:", df.columns.tolist())

# Drop the 'duration' column if it exists
if 'duration' in df.columns:
    df.drop(columns=['duration'], inplace=True)
    print(" 'duration' column dropped.")
else:
    print(" 'duration' column not found.")

# Show total columns after dropping
print(f" Total columns after dropping: {df.shape[1]}")
print(" Updated column names:", df.columns.tolist())

 Total columns before dropping: 6
 Column names: ['path', 'sentence', 'found', 'duration_sec', 'duration_ms', 'duration']
 'duration' column dropped.
 Total columns after dropping: 5
 Updated column names: ['path', 'sentence', 'found', 'duration_sec', 'duration_ms']


In [36]:
df.to_csv(csv_path, index=False)
print(" Updated CSV saved.")

 Updated CSV saved.


**Playing some audio files from final merged telugu audios folder**

In [41]:
audio_folder = "/home/jovyan/work/Datasets/final_merged_telugu_audios"
# List all .wav files
audio_files = [f for f in os.listdir(audio_folder) if f.endswith(".wav")]
# Count them
print(f" Total number of audio files: {len(audio_files)}")

 Total number of audio files: 162459


In [39]:
audio_files = sorted(os.listdir(audio_folder))[:5]  # first 5 files

for file in audio_files:
    print(f" Playing: {file}")
    display(Audio(filename=os.path.join(audio_folder, file)))

 Playing: 0.wav


 Playing: 1.wav


 Playing: 10.wav


 Playing: 100.wav


 Playing: 1000.wav


**For removing audio files from the folders**

In [42]:
class AudioDatasetManager:
    def __init__(self, folder_paths, merged_folder):
        self.folder_paths = folder_paths
        self.merged_folder = merged_folder

    def delete_audio_files(self):
        """Delete all .wav files in the merged folder."""
        deleted_count = 0
        for file in os.listdir(self.merged_folder):
            if file.endswith('.wav'):
                os.remove(os.path.join(self.merged_folder, file))
                deleted_count += 1
        return f"Deleted {deleted_count} .wav files."

# Define merged folder path
merged_audio_folder = "/home/jovyan/work/Datasets/telugu_audios_BA_final"

# Instantiate class and delete files
audio_manager = AudioDatasetManager([], merged_audio_folder)
print(audio_manager.delete_audio_files())


Deleted 152656 .wav files.


**Adding .wav extension the files which are not having .wav in the final merged telugu csv**

In [43]:
class TeluguDatasetProcessor:
    def __init__(self, input_csv_path, output_csv_path):
        self.input_csv_path = input_csv_path
        self.output_csv_path = output_csv_path
        self.df = None

    def load_dataset(self):
        self.df = pd.read_csv(self.input_csv_path)
        print(f" Loaded dataset with shape: {self.df.shape}")

    def fix_missing_wav_extensions(self):
        # Ensure all entries in 'path' column end with '.wav'
        self.df['path'] = self.df['path'].astype(str).apply(lambda x: x if x.endswith('.wav') else x + '.wav')
        print(" Fixed missing .wav extensions in 'path' column.")

    def save_updated_csv(self):
        self.df.to_csv(self.output_csv_path, index=False)
        print(f" Updated CSV saved as: {self.output_csv_path}")

    def analyze_dataset(self):
        print("\n Head:")
        print(self.df.head())

        print("\n Tail:")
        print(self.df.tail())

        print("\n Info:")
        print(self.df.info())

        print("\n Describe:")
        print(self.df.describe())

        print("\n Unique values per column:")
        print(self.df.nunique())

        total_duration_sec = self.df['duration_sec'].sum()
        print(f"\n Total duration (seconds): {total_duration_sec}")


base_path = "/home/jovyan/work/Datasets"
input_csv = os.path.join(base_path, "final_merged_telugu_dataset.csv")
output_csv = os.path.join(base_path, "final_merged_telugu_dataset_updated.csv")

# Create processor object
processor = TeluguDatasetProcessor(input_csv, output_csv)

# Process steps
processor.load_dataset()
processor.fix_missing_wav_extensions()
processor.save_updated_csv()
processor.analyze_dataset()


 Loaded dataset with shape: (162459, 5)
 Fixed missing .wav extensions in 'path' column.
 Updated CSV saved as: /home/jovyan/work/Datasets/final_merged_telugu_dataset_updated.csv

 Head:
                  path                                           sentence  \
0   telugu_audio_0.wav                                           నరకవాసి.   
1   telugu_audio_4.wav  ఆయనను ఇక్కడికి తీసుకురావటం దేనికమ్మా నా కాయనతో...   
2  telugu_audio_18.wav  బాబూ ఇది పద్మం కాదు అసిధారా చక్రం అన్నాడు నరకవ...   
3  telugu_audio_19.wav  చూశావా నా కివ్వటం ఇష్టంలేక ఆ మాట అంటున్నావు అన...   
4  telugu_audio_22.wav  ఆయనను చూడగానే మిత్రవిందకుడు ఏడుస్తూ స్వామీ కరు...   

  found  duration_sec  duration_ms  
0   yes      2.162000    2162.0000  
1   yes      8.268875    8268.8750  
2   yes      6.271937    6271.9375  
3   yes      7.897375    7897.3750  
4   yes     13.493375   13493.3750  

 Tail:
              path                                           sentence found  \
162454  160350.wav                     నా

**Checking merged audio and csv files for telugu and tamil**

In [44]:
class AudioDatasetValidator:
    def __init__(self, csv_path, audio_folder, language):
        self.csv_path = csv_path
        self.audio_folder = audio_folder
        self.language = language
        self.df = None
        self.missing_files = []
    
    def load_csv(self):
        self.df = pd.read_csv(self.csv_path)
        print(f"\n [{self.language}] Loaded CSV: {self.csv_path}")
        print(f"Total rows: {len(self.df)}")

    def check_audio_files_exist(self):
        all_audio_files = set(os.listdir(self.audio_folder))
        self.missing_files = []

        for audio_file in self.df['path'].astype(str):
            if audio_file not in all_audio_files:
                self.missing_files.append(audio_file)

        found = len(self.df) - len(self.missing_files)
        print(f" [{self.language}] Files Found: {found}")
        print(f" [{self.language}] Missing Files: {len(self.missing_files)}")

        if self.missing_files:
            print(f"Some missing files (first 5): {self.missing_files[:5]}")

    def show_csv_statistics(self):
        print(f"\n [{self.language}] CSV Statistics:")
        print("\n Head:")
        print(self.df.head())
        print("\n Tail:")
        print(self.df.tail())
        print("\n Info:")
        print(self.df.info())
        print("\n Describe:")
        print(self.df.describe())
        print("\n Unique counts per column:")
        print(self.df.nunique())
        print(f"\n Total duration (sec): {self.df['duration_sec'].sum():.2f}")
        print(f" Total duration (ms): {self.df['duration_ms'].sum():.2f}")

    def count_audio_files_by_type(self):
        total_files = os.listdir(self.audio_folder)
        wav_files = [f for f in total_files if f.endswith('.wav')]
        mp3_files = [f for f in total_files if f.endswith('.mp3')]

        print(f"\n [{self.language}] Audio Folder: {self.audio_folder}")
        print(f" Total files: {len(total_files)}")
        print(f" .mp3 files: {len(mp3_files)}")
        print(f" .wav files: {len(wav_files)}")


# Tamil Dataset
tamil_validator = AudioDatasetValidator(
    csv_path="/home/jovyan/work/Datasets/final_merged_tamil_dataset.csv",
    audio_folder="/home/jovyan/work/Datasets/final_merged_tamil_audios",
    language="Tamil"
)

tamil_validator.load_csv()
tamil_validator.check_audio_files_exist()
tamil_validator.show_csv_statistics()
tamil_validator.count_audio_files_by_type()

# Telugu Dataset
telugu_validator = AudioDatasetValidator(
    csv_path="/home/jovyan/work/Datasets/final_merged_telugu_dataset_updated.csv",
    audio_folder="/home/jovyan/work/Datasets/final_merged_telugu_audios",
    language="Telugu"
)

telugu_validator.load_csv()
telugu_validator.check_audio_files_exist()
telugu_validator.show_csv_statistics()
telugu_validator.count_audio_files_by_type()


 [Tamil] Loaded CSV: /home/jovyan/work/Datasets/final_merged_tamil_dataset.csv
Total rows: 259183
 [Tamil] Files Found: 259183
 [Tamil] Missing Files: 0

 [Tamil] CSV Statistics:

 Head:
                           path  \
0  common_voice_ta_24959711.mp3   
1  common_voice_ta_24959713.mp3   
2  common_voice_ta_24959750.mp3   
3  common_voice_ta_24959754.mp3   
4  common_voice_ta_24959762.mp3   

                                            sentence found  duration_sec  \
0              பாலாஜி என்பது பாலன் என்ற பொருளுடையது.   Yes         6.696   
1  சீக்கிரத்தில் ஐநூறு ரூபாய் செலவழித்து அவரது மு...   Yes         7.380   
2                   பகலில் இயங்கிக் கொண்டே இருப்பர்.   Yes         3.960   
3                          மிகுதியும் பயன் விளையும்.   Yes         3.456   
4      புதுப்புனலில் பாய்ந்து நீந்தி விளையாடுகிறாள்.   Yes         4.428   

   duration_ms  
0       6696.0  
1       7380.0  
2       3960.0  
3       3456.0  
4       4428.0  

 Tail:
                        path  \
25

**Removing data which are having duration > 15sec and updating audio folders and csv for tamil**

In [45]:
class TamilAudioDurationFilter:
    def __init__(self, input_csv, original_audio_folder, output_csv, output_audio_folder):
        self.input_csv = input_csv
        self.original_audio_folder = original_audio_folder
        self.output_csv = output_csv
        self.output_audio_folder = output_audio_folder
        self.df = None
        self.filtered_df = None

    def load_csv(self):
        self.df = pd.read_csv(self.input_csv)
        print(f" Loaded CSV: {self.input_csv} with shape {self.df.shape}")

    def filter_by_duration(self, max_duration_sec=15.0):
        self.filtered_df = self.df[self.df['duration_sec'] <= max_duration_sec].copy()
        print(f" Filtered rows with duration <= {max_duration_sec} sec: {self.filtered_df.shape[0]} rows retained")

    def save_filtered_csv(self):
        self.filtered_df.to_csv(self.output_csv, index=False)
        print(f" Saved filtered CSV to: {self.output_csv}")

    def copy_filtered_audio_files(self):
        os.makedirs(self.output_audio_folder, exist_ok=True)
        total_copied = 0

        for file in self.filtered_df['path']:
            src = os.path.join(self.original_audio_folder, file)
            dst = os.path.join(self.output_audio_folder, file)
            if os.path.exists(src):
                shutil.copy2(src, dst)
                total_copied += 1

        print(f" Copied {total_copied} audio files to {self.output_audio_folder}")

    def validate_audio_files(self):
        audio_files = set(os.listdir(self.output_audio_folder))
        missing = []

        for file in self.filtered_df['path']:
            if file not in audio_files:
                missing.append(file)

        print(f" Found: {len(self.filtered_df) - len(missing)}")
        print(f" Missing: {len(missing)}")
        if missing:
            print("Some missing files (first 5):", missing[:5])

    def show_statistics(self):
        print("\n CSV Statistics for Filtered Dataset:")
        print("\n Head:")
        print(self.filtered_df.head())

        print("\n Tail:")
        print(self.filtered_df.tail())

        print("\n Info:")
        print(self.filtered_df.info())

        print("\n Describe:")
        print(self.filtered_df.describe())

        print("\n Unique counts per column:")
        print(self.filtered_df.nunique())

        print(f"\n Total duration (sec): {self.filtered_df['duration_sec'].sum()}")
        print(f" Total duration (ms): {self.filtered_df['duration_ms'].sum()}")


base_path = "/home/jovyan/work/Datasets"

processor = TamilAudioDurationFilter(
    input_csv=os.path.join(base_path, "final_merged_tamil_dataset.csv"),
    original_audio_folder=os.path.join(base_path, "final_merged_tamil_audios"),
    output_csv=os.path.join(base_path, "final_merged_tamil_dataset_updated.csv"),
    output_audio_folder=os.path.join(base_path, "final_merged_tamil_audios_updated")
)

processor.load_csv()
processor.filter_by_duration(max_duration_sec=15.0)
processor.save_filtered_csv()
processor.copy_filtered_audio_files()
processor.validate_audio_files()
processor.show_statistics()


 Loaded CSV: /home/jovyan/work/Datasets/final_merged_tamil_dataset.csv with shape (259183, 5)
 Filtered rows with duration <= 15.0 sec: 259137 rows retained
 Saved filtered CSV to: /home/jovyan/work/Datasets/final_merged_tamil_dataset_updated.csv
 Copied 259137 audio files to /home/jovyan/work/Datasets/final_merged_tamil_audios_updated
 Found: 259137
 Missing: 0

 CSV Statistics for Filtered Dataset:

 Head:
                           path  \
0  common_voice_ta_24959711.mp3   
1  common_voice_ta_24959713.mp3   
2  common_voice_ta_24959750.mp3   
3  common_voice_ta_24959754.mp3   
4  common_voice_ta_24959762.mp3   

                                            sentence found  duration_sec  \
0              பாலாஜி என்பது பாலன் என்ற பொருளுடையது.   Yes         6.696   
1  சீக்கிரத்தில் ஐநூறு ரூபாய் செலவழித்து அவரது மு...   Yes         7.380   
2                   பகலில் இயங்கிக் கொண்டே இருப்பர்.   Yes         3.960   
3                          மிகுதியும் பயன் விளையும்.   Yes         3.456 